In [1]:
# Import required libraries
import os
import cv2
import numpy as np
from scipy.ndimage import convolve
from scipy.signal import wiener
from skimage import img_as_float
from skimage import img_as_float, img_as_ubyte

## Compute Saliency based on Laplacian and Wiener filter

In [ ]:

# Function to create a directory if it doesn't exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Specify the folder containing the videos
input_folder = '/replay_dataset_saliency/test/real'  # Replace with the actual folder path
output_folder = '/replay_dataset_saliency/test/real_com_sil'  # Replace with the output folder path

# Get a list of all video files in the folder
video_files = [f for f in os.listdir(input_folder) if f.endswith(('.avi', '.mp4', '.mov'))]  # Change the extensions as necessary

# Loop through each video file
for video_file in video_files:
    # Read the video file
    video_path = os.path.join(input_folder, video_file)
    video = cv2.VideoCapture(video_path)
    
    # Create a folder for saving images
    video_name = os.path.splitext(video_file)[0]
    create_directory(os.path.join(output_folder, video_name))
    
    # Read the first frame
    ret, first_frame = video.read()
    if not ret:
        print(f"Failed to read the video file: {video_file}")
        continue

    # Initialize variables for averaging
    sum_frames = np.float64(first_frame)
    frame_count = 1
    
    # Read and accumulate frames from the video
    while True:
        ret, frame = video.read()
        if not ret:
            break
        sum_frames += np.float64(frame)
        frame_count += 1

    # Calculate the average frame
    average_frame = np.uint8(sum_frames / frame_count)

    # Normalize the images
    first_frame = img_as_float(first_frame)
    average_frame = img_as_float(average_frame)

    # Define the Laplacian filter
    Laplacian = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

    # Apply the Laplacian filter to the first image
    noisyR, noisyG, noisyB = cv2.split(first_frame)
    denoisedR = convolve(noisyR, Laplacian)
    denoisedG = convolve(noisyG, Laplacian)
    denoisedB = convolve(noisyB, Laplacian)
    Filtering1 = cv2.merge([denoisedR, denoisedG, denoisedB])

    # Apply Wiener filter to the first image
    noisyR = noisyR + np.random.normal(0, 0.005, noisyR.shape)
    denoisedR = wiener(noisyR, (15, 15))
    noisyG = noisyG + np.random.normal(0, 0.005, noisyG.shape)
    denoisedG = wiener(noisyG, (15, 15))
    noisyB = noisyB + np.random.normal(0, 0.005, noisyB.shape)
    denoisedB = wiener(noisyB, (15, 15))
    Filtering2 = cv2.merge([denoisedR, denoisedG, denoisedB])

    # Saliency estimation for the first image
    Saliency1 = (Filtering1 - Filtering2) ** 2
    d1 = first_frame - Filtering1

    # Apply the Laplacian filter to the second image (average frame)
    noisyR, noisyG, noisyB = cv2.split(average_frame)
    denoisedR = convolve(noisyR, Laplacian)
    denoisedG = convolve(noisyG, Laplacian)
    denoisedB = convolve(noisyB, Laplacian)
    b2 = cv2.merge([denoisedR, denoisedG, denoisedB])

    # Apply Wiener filter to the second image
    noisyR = noisyR + np.random.normal(0, 0.005, noisyR.shape)
    denoisedR = wiener(noisyR, (15, 15))
    noisyG = noisyG + np.random.normal(0, 0.005, noisyG.shape)
    denoisedG = wiener(noisyG, (15, 15))
    noisyB = noisyB + np.random.normal(0, 0.005, noisyB.shape)
    denoisedB = wiener(noisyB, (15, 15))
    Image2 = cv2.merge([denoisedR, denoisedG, denoisedB])

    # Saliency estimation for the second image
    Saliency2 = (b2 - Image2) ** 2
    d2 = average_frame - b2

    # Weights normalization
    weight1 = Saliency1 / (Saliency1 + Saliency2 + 1e-6)  # Adding a small value to avoid division by zero
    weight2 = Saliency2 / (Saliency1 + Saliency2 + 1e-6)

    Finalimag1 = weight1 * d1 + weight2 * d2
    finalimage2 = 0.5 * Filtering1 + 0.5 * b2
    Fusedframe = Finalimag1 + finalimage2

    # Clip and normalize the fused frame before saving
    Fusedframe = np.clip(Fusedframe, 0, 1)
    Fusedframe = img_as_ubyte(Fusedframe)

    # Save the fused frame
    fused_frame_filename = os.path.join(output_folder, video_name, f"{video_name}.jpg")
    cv2.imwrite(fused_frame_filename, Fusedframe)

    print(f"First frame and average frame processed for saliency-based fusion: {video_file}")

print("All videos processed.")